In [1]:
import pandas as pd
import os, gc, re
from tqdm import tqdm

In [2]:
df_cats = pd.DataFrame(columns=['file', 'category'])
df = pd.DataFrame(columns=['file', 'message'])

In [3]:
for fol in tqdm(os.listdir('data')):
    for file in os.listdir(f'data/{fol}'):
        if '.cats' in file:
            with open(f'data/{fol}/{file}', 'r') as st:
                str1 = st.read()
            str2 = str1.split('\n')[0].split(',')[1]
            temp = {}
            temp['file'] = file.replace('.cats', '')
            temp['category'] = str2
            df_cats = pd.concat([df_cats, pd.DataFrame([temp])], ignore_index=True)
        elif '.txt' in file:
            with open(f'data/{fol}/{file}', 'r') as st:
                str1 = st.read()
            temp = {}
            temp['file'] = file.replace('.txt', '')
            temp['message'] = str1
            df = pd.concat([df, pd.DataFrame([temp])], ignore_index=True)

  0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 8/8 [00:20<00:00,  2.51s/it]


In [4]:
df_cats

,file,category
0,10425,1
1,106296,1
2,106298,1
3,106588,1
4,106590,1
...,...,...
1697,176657,8
1698,176744,8
1699,177851,8
1700,232322,8


In [5]:
df

,file,message
0,10425,Message-ID: <197504.1075840201539.JavaMail.eva...
1,106296,Message-ID: <11991339.1075842536086.JavaMail.e...
2,106298,Message-ID: <7106753.1075842536132.JavaMail.ev...
3,106588,Message-ID: <21267718.1075863331587.JavaMail.e...
4,106590,Message-ID: <20866019.1075863331634.JavaMail.e...
...,...,...
1697,176657,Message-ID: <31804067.1075849874410.JavaMail.e...
1698,176744,Message-ID: <6453004.1075846234689.JavaMail.ev...
1699,177851,Message-ID: <14103554.1075858884564.JavaMail.e...
1700,232322,Message-ID: <4760206.1075853182821.JavaMail.ev...


In [6]:
def info_part(i):
    """split infomation part out"""
    return i.split('\n\n', 1)[0]
def content_part(i):
    """split content part out"""
    return i.split('\n\n', 1)[1]
df['pre_info'] = df.message.map(info_part)
df['content'] = df.message.map(content_part)
df['test_true'] = True

words2split = ['Message-ID: ', 'Date: ', 'From: ', 'To: ', 'Subject: ', 'Cc: ', 'Mime-Version: ', 'Content-Type: ',
               'Content-Transfer-Encoding: ', 'Bcc: ', 'X-From: ', 'X-To: ', 'X-cc: ', 'X-bcc: ', 'X-Folder: ', 'X-Origin: ',
               'X-FileName: ']
features_naming = [i[:-2] for i in words2split]
split_condition = '|'.join(words2split)



In [7]:
# Some emails' subject confuse the string-spliting function, so I make a little change
def duplicated_info(i):
    return i.replace(' Date: ', ' Date- ').replace(' Subject: ', ' Subject2: ').replace(' To: ',
                    ' To- ').replace(' (Subject: ', ' (Subject- ')
df['pre_info'] = df['pre_info'].map(duplicated_info)

# let's check how many categories are there in these emails
def num_part(i):
    return len(re.split(split_condition, i))
df['num_info'] = df['pre_info'].map(num_part)

# around 20k emails do not have the 'To: ' category, so I add one
def add_to(i):
    return i.replace('\nSubject: ', '\nTo: \nSubject: ')
temp_condition = (df['num_info'] == 17) | (df['num_info'] == 15)
df.loc[temp_condition, 'pre_info'] = df.loc[temp_condition, 'pre_info'].map(add_to)


# similar way to deal with the "Cc:" and "Bcc:" categories
temp_condition = (df['num_info'] == 16) | (df['num_info'] == 15)
def add_bcc(i):
    return i.replace('\nX-From: ', '\nBcc: \nX-From: ')
df.loc[temp_condition, 'pre_info'] = df.loc[temp_condition, 'pre_info'].map(add_bcc)
def add_cc(i):
    return i.replace('\nMime-Version: ', '\nCc: \nMime-Version: ')
df.loc[temp_condition, 'pre_info'] = df.loc[temp_condition, 'pre_info'].map(add_cc)

In [8]:
df['num_info'] = df['pre_info'].map(num_part)
df['num_info'].value_counts()

18    1702
Name: num_info, dtype: int64

In [9]:
global feature_idx

def info_split(i):
    # Split the i th part out and remove \n for the feature
    split_results = re.split(split_condition, i)
    if len(split_results) > feature_idx + 1:
        return split_results[feature_idx + 1][:-2]
    else:
        return None

def info_split_last(i):
    # No need to remove \n for the last category -- X-FileName
    split_results = re.split(split_condition, i)
    if len(split_results) > feature_idx + 1:
        return split_results[feature_idx + 1]
    else:
        return None

# Loop through each feature index
for feature_idx in range(len(words2split)):
    if feature_idx != len(words2split) - 1:
        df[features_naming[feature_idx]] = df['pre_info'].map(info_split)
    else:
        df[features_naming[feature_idx]] = df['pre_info'].map(info_split_last)

In [10]:
df['Content-Transfer-Encoding'].value_counts()

7bi                1583
quoted-printabl     119
Name: Content-Transfer-Encoding, dtype: int64

In [11]:
df_remove2 = df.loc[df['Content-Transfer-Encoding'] == 'text/plain; charset=us-asci']
df = df.loc[df['Content-Transfer-Encoding'] != 'text/plain; charset=us-asci']

In [12]:
df.loc[df["content"].str.contains("-------------"), "content"]

0       ---------------------- Forwarded by Steven J K...
2       Julia and Steve--here are some questions I've ...
3       Julia and Steve--here are some questions I've ...
7       Heads up.  This is not to be discussed in adva...
8       FYI.\n\nJDS\n---------------------- Forwarded ...
                              ...                        
1688    ---------------------- Forwarded by Steven J K...
1691    ---------------------- Forwarded by Steven J K...
1692    ----- Forwarded by Steven J Kean/NA/Enron on 0...
1695    ---------------------- Forwarded by Steven J K...
1701    ---------------------- Forwarded by Mary Hain/...
Name: content, Length: 577, dtype: object

In [13]:
def split_other_content(i):
    """split other forms of contents out"""
    return i.split('-------------', 1)[0]
df["has_other_content"] = df["content"].str.contains("-------------")
df["if_forwarded"] = df["content"].str.contains("------------- Forwarded")
df['content'] = df.content.map(split_other_content)

In [14]:
df = df.drop(['pre_info','test_true', 'num_info'], axis = 1).set_index("file")
df.to_csv("emails_cleaned.csv")

In [15]:
df

,message,content,Message-ID,Date,From,To,Subject,Cc,Mime-Version,Content-Type,...,Bcc,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,has_other_content,if_forwarded
file,,,,,,,,,,,,,,,,,,,,,
10425,Message-ID: <197504.1075840201539.JavaMail.eva...,,<197504.1075840201539.JavaMail.evans@thyme,"Mon, 18 Oct 1999 01:47:00 -0700 (PDT",steven.kean@enron.co,"mark.schroeder@enron.com, kenneth.lay@enron.co...",Translation of article,,1.,text/plain; charset=us-asci,...,,Steven J Kea,"Mark Schroeder, Kenneth Lay, Joseph W Sutton, ...",,,\Kenneth_Lay_Dec2000\Notes Folders\All document,LAY-,klay.nsf,True,True
106296,Message-ID: <11991339.1075842536086.JavaMail.e...,In anticipation of potential litigation involv...,<11991339.1075842536086.JavaMail.evans@thyme,"Mon, 5 Mar 2001 08:23:00 -0800 (PST",drew.fossum@enron.co,darrell.schoolcraft@enron.co,TW Gas Sales: PRIVILEGED AND CONFIDENTIAL ATTO...,"danny.mccarty@enron.com, steven.harris@enron.c...",1.,text/plain; charset=us-asci,...,"danny.mccarty@enron.com, steven.harris@enron.c...",Drew Fossu,Darrell Schoolcraf,"Danny McCarty, Steven Harris, Kevin Hyat",,\Drew_Fossum_Dec2000_June2001_2\Notes Folders\...,FOSSUM-,dfossum.nsf,False,False
106298,Message-ID: <7106753.1075842536132.JavaMail.ev...,Julia and Steve--here are some questions I've ...,<7106753.1075842536132.JavaMail.evans@thyme,"Tue, 6 Mar 2001 00:59:00 -0800 (PST",drew.fossum@enron.co,"julia.white@enron.com, steven.january@enron.co",TW Gas Sales: PRIVILEGED AND CONFIDENTIAL ATTO...,darrell.schoolcraft@enron.co,1.,text/plain; charset=us-asci,...,darrell.schoolcraft@enron.co,Drew Fossu,"Julia White, Steven Januar",Darrell Schoolcraf,,\Drew_Fossum_Dec2000_June2001_2\Notes Folders\...,FOSSUM-,dfossum.nsf,True,True
106588,Message-ID: <21267718.1075863331587.JavaMail.e...,Julia and Steve--here are some questions I've ...,<21267718.1075863331587.JavaMail.evans@thyme,"Tue, 6 Mar 2001 11:59:00 -0800 (PST",drew.fossum@enron.co,"julia.white@enron.com, steven.january@enron.co",TW Gas Sales: PRIVILEGED AND CONFIDENTIAL ATTO...,darrell.schoolcraft@enron.co,1.,text/plain; charset=us-asci,...,darrell.schoolcraft@enron.co,Drew Fossu,"Julia White <Julia White/ET&S/Enron@ENRON>, St...",Darrell Schoolcraft <Darrell Schoolcraft/ET&S/...,,"\DFOSSUM (Non-Privileged)\Fossum, Drew\'Sent Mai",Fossum-,DFOSSUM (Non-Privileged).pst,True,True
106590,Message-ID: <20866019.1075863331634.JavaMail.e...,In anticipation of potential litigation involv...,<20866019.1075863331634.JavaMail.evans@thyme,"Mon, 5 Mar 2001 19:23:00 -0800 (PST",drew.fossum@enron.co,darrell.schoolcraft@enron.co,TW Gas Sales: PRIVILEGED AND CONFIDENTIAL ATTO...,"danny.mccarty@enron.com, steven.harris@enron.c...",1.,text/plain; charset=us-asci,...,"danny.mccarty@enron.com, steven.harris@enron.c...",Drew Fossu,Darrell Schoolcraft <Darrell Schoolcraft/ET&S/...,Danny McCarty <Danny McCarty/ET&S/Enron@Enron>...,,"\DFOSSUM (Non-Privileged)\Fossum, Drew\'Sent Mai",Fossum-,DFOSSUM (Non-Privileged).pst,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176657,Message-ID: <31804067.1075849874410.JavaMail.e...,", Palmer",<31804067.1075849874410.JavaMail.evans@thyme,"Mon, 18 Jun 2001 06:30:00 -0700 (PDT",steven.kean@enron.co,,"Lay, Palmer and isakof",,1.,text/plain; charset=us-asci,...,,Steven J Kea,,,,\Steven_Kean_Nov2001_1\Notes Folders\All document,KEAN-,skean.nsf,False,False
176744,Message-ID: <6453004.1075846234689.JavaMail.ev...,test,<6453004.1075846234689.JavaMail.evans@thyme,"Fri, 15 Sep 2000 09:06:00 -0700 (PDT",steven.kean@enron.co,dave.martinez@enron.co,,,1.,text/plain; charset=us-asci,...,,Steven J Kea,Dave Martine,,,\Steven_Kean_Dec2000_1\Notes Folders\Archiving...,KEAN-,skean.nsf,False,False
177851,Message-ID: <14103554.1075858884564.JavaMail.e...,testing,<14103554.1075858884564.JavaMail.evans@thyme,"Tue, 26 Jun 2001 18:51:00 -0700 (PDT",steven.kean@enron.co,maureen.mcvicker@enron.co,Tes,,1.,text/plain; charset=us-asci,...,,Steven J Kea,Maureen Mc

In [16]:
data = df.reset_index()[['file', 'content']]
data = data.merge(df_cats, on='file', how='left')

In [21]:
data = data[['content', 'category']]

In [28]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [29]:
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Apply stopword removal to the 'content' column
data['content'] = data['content'].apply(remove_stopwords)


C:\Users\lenovo\AppData\Local\Temp/ipykernel_3672/3465068220.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['content'] = data['content'].apply(remove_stopwords)


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import time

In [34]:
vectorization_methods = {
    'Bag of Words': CountVectorizer(min_df=5, max_features=5000),
    'Bag of Words (Bi-gram)': CountVectorizer(min_df=5, max_features=5000, ngram_range=(2,2)),
    'TF-IDF': TfidfVectorizer(max_features=500)
}

# Define classifiers
classifiers = {
    'Gaussian NB': GaussianNB(),
    'Multinomial NB': MultinomialNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'SVM': SVC(),
    'Adaboost': AdaBoostClassifier(),
    'ANN': MLPClassifier()
}

In [35]:
results = []

# Iterate over vectorization methods and classifiers
for vec_name, vectorizer in tqdm(vectorization_methods.items()):
    X = vectorizer.fit_transform(data['content'])
    y = data['category']
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    for clf_name, classifier in tqdm(classifiers.items()):
        start_time = time.time()
        if clf_name == 'Gaussian NB':
            X_train_dense = X_train.toarray()
            X_val_dense = X_val.toarray()
            classifier.fit(X_train_dense, y_train)
            y_val_pred = classifier.predict(X_val_dense)
        else:
            classifier.fit(X_train, y_train)
            y_val_pred = classifier.predict(X_val)

        end_time = time.time()

        accuracy = accuracy_score(y_val, y_val_pred)
        f1 = f1_score(y_val, y_val_pred, average='weighted')
        time_taken = end_time - start_time

        results.append({
            'Vectorization Method': vec_name,
            'Classifier': clf_name,
            'Accuracy': accuracy,
            'F1 Score': f1,
            'Time Taken (s)': time_taken
        })

  0%|          | 0/3 [00:00<?, ?it/s]

 67%|██████▋   | 2/3 [00:50<00:26, 26.70s/it]c:\Users\lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
100%|██████████| 3/3 [00:56<00:00, 18.90s/it]


In [36]:
# Display results
results_df = pd.DataFrame(results)
print(results_df)

      Vectorization Method      Classifier  Accuracy  F1 Score  Time Taken (s)
0             Bag of Words     Gaussian NB  0.504399  0.503783        0.277832
1             Bag of Words  Multinomial NB  0.486804  0.495127        0.003984
2             Bag of Words   Decision Tree  0.580645  0.571932        0.182595
3             Bag of Words             SVM  0.530792  0.372164        1.251671
4             Bag of Words        Adaboost  0.513196  0.501272        0.549267
5             Bag of Words             ANN  0.662757  0.642625       14.511469
6   Bag of Words (Bi-gram)     Gaussian NB  0.434018  0.447194        0.243630
7   Bag of Words (Bi-gram)  Multinomial NB  0.384164  0.391418        0.004599
8   Bag of Words (Bi-gram)   Decision Tree  0.460411  0.459381        0.086737
9   Bag of Words (Bi-gram)             SVM  0.545455  0.397060        0.525878
10  Bag of Words (Bi-gram)        Adaboost  0.507331  0.405831        0.304574
11  Bag of Words (Bi-gram)             ANN  0.524927

In [37]:
results_df

,Vectorization Method,Classifier,Accuracy,F1 Score,Time Taken (s)
0,Bag of Words,Gaussian NB,0.504399,0.503783,0.277832
1,Bag of Words,Multinomial NB,0.486804,0.495127,0.003984
2,Bag of Words,Decision Tree,0.580645,0.571932,0.182595
3,Bag of Words,SVM,0.530792,0.372164,1.251671
4,Bag of Words,Adaboost,0.513196,0.501272,0.549267
5,Bag of Words,ANN,0.662757,0.642625,14.511469
6,Bag of Words (Bi-gram),Gaussian NB,0.434018,0.447194,0.243630
7,Bag of Words (Bi-gram),Multinomial NB,0.384164,0.391418,0.004599
8,Bag of Words (Bi-gram),Decision Tree,0.460411,0.459381,0.086737
9,Bag of Words (Bi-gram),SVM,0.545455,0.397060,0.525878
